# Exploration of Victorian Accident Data

## Introduction

This is an semi-shallow analysis of Road Vehicle accident data from Australia, Victoria.

source: https://discover.data.vic.gov.au/dataset/crash-stats-data-extract accessed 13-09-2023   

Tables

- accident (basic accident details, time, severity, location)

- person (person based details, age, sex etc)

- vehicle (vehicle based data, vehicle type, make etc)

- accident_event (sequence of events e.g. left road, rollover, caught fire)

- road_surface_cond (whether road was wet, dry, icy etc)

- atmospheric_cond (rain, winds etc)

- sub_dca (detailed codes describing accident)

- accident_node (master location table - NB subset of accident table)

- Node Table with Lat/Long references

for the purposes of this analysis, we will focus on the accident table.


In [9]:
### Quick Overview of the Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [3]:
df = pd.read_csv('data\ACCIDENT.csv')

C:\Users\oweng\AppData\Local\Temp\ipykernel_6212\2967269902.py:1: DtypeWarning: Columns (10,13) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data\ACCIDENT.csv')


In [5]:
pd.set_option('display.max_columns', None)
df.head()

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,EDITION,PAGE,GRID_REFERENCE_X,GRID_REFERENCE_Y,LIGHT_CONDITION,Light Condition Desc,NODE_ID,NO_OF_VEHICLES,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
0,T20060000010,13/01/2006,12:42:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,40,91A,G,7,1,Day,43078,3,6,0,1,0,5,1,1,Cross intersection,3,60
1,T20060000018,13/01/2006,19:10:00,1,Collision with vehicle,6,Friday,113,RIGHT NEAR (INTERSECTIONS ONLY),MEL,40,91,H,8,1,Day,29720,2,4,0,1,0,3,1,2,T intersection,3,70
2,T20060000022,14/01/2006,12:10:00,7,Fall from or in moving vehicle,7,Saturday,190,FELL IN/FROM VEHICLE,MEL,40,169,C,11,1,Day,203074,1,2,1,0,0,1,1,5,Not at intersection,2,100
3,T20060000023,14/01/2006,11:49:00,1,Collision with vehicle,7,Saturday,130,REAR END(VEHICLES IN SAME LANE),MEL,40,88,J,8,1,Day,55462,2,2,1,0,0,1,1,2,T intersection,2,80
4,T20060000026,14/01/2006,10:45:00,1,Collision with vehicle,7,Saturday,121,RIGHT THROUGH,MEL,40,169,G,5,1,Day,202988,2,3,0,3,0,0,1,5,Not at intersection,3,50


In [8]:
df.tail()

,ACCIDENT_NO,ACCIDENTDATE,ACCIDENTTIME,ACCIDENT_TYPE,Accident Type Desc,DAY_OF_WEEK,Day Week Description,DCA_CODE,DCA Description,DIRECTORY,EDITION,PAGE,GRID_REFERENCE_X,GRID_REFERENCE_Y,LIGHT_CONDITION,Light Condition Desc,NODE_ID,NO_OF_VEHICLES,NO_PERSONS,NO_PERSONS_INJ_2,NO_PERSONS_INJ_3,NO_PERSONS_KILLED,NO_PERSONS_NOT_INJ,POLICE_ATTEND,ROAD_GEOMETRY,Road Geometry Desc,SEVERITY,SPEED_ZONE
203703,T20200019239,1/11/2020,12:11:00,1,Collision with vehicle,0,Sunday,142,LEAVING PARKING,MEL,46,44,B,12,1,Day,665835,2,4,1,0,0,3,1,5,Not at intersection,2,50
203704,T20200019247,1/11/2020,15:30:00,4,Collision with a fixed object,1,Sunday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,46,468,A,9,1,Day,665912,1,2,2,0,0,0,1,5,Not at intersection,2,999
203705,T20200019250,1/11/2020,18:00:00,1,Collision with vehicle,0,Sunday,116,LEFT NEAR (INTERSECTIONS ONLY),MEL,40,10,D,1,1,Day,226603,2,2,1,0,0,1,1,1,Cross intersection,2,60
203706,T20200019253,1/11/2020,12:00:00,6,Vehicle overturned (no collision),1,Sunday,180,OFF CARRIAGEWAY ON RIGHT BEND,VCD,9,50,J,9,1,Day,665836,1,1,1,0,0,0,1,5,Not at intersection,2,80
203707,T20200019417,4/11/2020,01:30:00,4,Collision with a fixed object,3,Wednesday,171,LEFT OFF CARRIAGEWAY INTO OBJECT/PARKED VEHICL...,MEL,46,88,F,8,3,Dark Street lights on,666096,1,1,1,0,0,0,1,5,Not at intersection,2,80


In [7]:
def check_missing(df):
    for col in df.columns:
        if df[col].isnull().sum() > 0:
            print(col, df[col].isnull().sum())
            
check_missing(df)

DIRECTORY 2348
EDITION 2348
PAGE 2348
GRID_REFERENCE_X 2348
GRID_REFERENCE_Y 2348


In [10]:
def check_dtypes(df):
    for col in df.columns:
        print(col, df[col].dtype)
        
check_dtypes(df)

ACCIDENT_NO object
ACCIDENTDATE object
ACCIDENTTIME object
ACCIDENT_TYPE int64
Accident Type Desc object
DAY_OF_WEEK int64
Day Week Description object
DCA_CODE int64
DCA Description object
DIRECTORY object
EDITION object
PAGE object
GRID_REFERENCE_X object
GRID_REFERENCE_Y object
LIGHT_CONDITION int64
Light Condition Desc object
NODE_ID int64
NO_OF_VEHICLES int64
NO_PERSONS int64
NO_PERSONS_INJ_2 int64
NO_PERSONS_INJ_3 int64
NO_PERSONS_KILLED int64
NO_PERSONS_NOT_INJ int64
POLICE_ATTEND int64
ROAD_GEOMETRY int64
Road Geometry Desc object
SEVERITY int64
SPEED_ZONE int64
